In [ ]:
# XOU CLEANED

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from math import exp, sqrt, log
from scipy.stats import ks_2samp
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
def generate_cleaned_XOU_path(gamma, sigma, phi, X0, N, dt, seed=300):
    np.random.seed(seed)
    X = np.zeros(N +1)
    X[0] = X0
    for t in range(1, N + 1):
        dW = np.random.normal(0, np.sqrt(dt))
        Y_t_prev = log(X[t-1])
        Y_t = (Y_t_prev * exp(-gamma * dt) + (phi - (sigma**2) / (2 * gamma)) * (1 - exp(-gamma * dt)) + 
               sigma * sqrt((1 / (2 * gamma)) * (1 - exp(-2 * gamma * dt))) * dW)
        X[t] = exp(Y_t)
    return X
filename = 'Historical_Prices_SBU_Dec_2024.csv'
df = pd.read_csv(filename)
df['Date'] = pd.to_datetime(df['Date'])
fixed_end_day = 1480
prediction_horizons = [100, 50, 30]
num_scenarios = 10000
dt = 1
seed = 300
window_range = range(100, 310,10)
all_horizon_summaries = {}
for horizon in prediction_horizons:
    print(f"\n" + "="*100)
    print(f"FULL DATA REPORT FOR {horizon}-DAY HORIZON")
    print("="*100)
    actual_prices = df['Close'].values[fixed_end_day - 1 : fixed_end_day + horizon]
    prediction_dates = df['Date'].values[fixed_end_day - 1 : fixed_end_day + horizon]
    initial_date = pd.to_datetime(prediction_dates[0])
    print(f"INITIAL DATE (Day 0): {initial_date.strftime('%Y-%m-%d')}")
    grid_results = []
    for w in window_range:
        start_calib = (fixed_end_day - 1) - w
        M_raw = df['Close'].values[start_calib : fixed_end_day - 1]
        res_bin = [1 if M_raw[i] != M_raw[i+1] else 0 for i in range(len(M_raw) - 1)]
        pairs = [f"{res_bin[i]}{res_bin[i+1]}" for i in range(len(res_bin) - 1)]
        counts = Counter(pairs)
        phat = counts.get("00",0) / (counts.get("00", 0) + counts.get("01", 0)) 
        qhat = counts.get("11",0) / (counts.get("10", 0) + counts.get("11", 0)) 
        M_clean = [M_raw[0]]
        for i in range(1, len(M_raw)):
            if M_raw[i] != M_raw[i-1]: M_clean.append(M_raw[i])
        log_M = np.log(np.array(M_clean))
        n = len(log_M) - 1
        bb1, bb2 = log_M[:-1], log_M[1:]
        Y1, Y2 = np.sum(bb2), np.sum(bb1)
        Y11, Y12, Y22 = np.sum(bb2**2), np.sum(bb1 * bb2),  np.sum(bb1**2)
        phiperp = ((Y2 * Y11) - (Y1 * Y12)) / ((n *(Y11 - Y12)) - ((Y1**2) - (Y1 * Y2)))
        alpha = (Y12 -(phiperp * Y2) - (phiperp * Y1) + (n * (phiperp**2))) / \
                (Y11 - (2 * phiperp * Y1) + (n * (phiperp**2)))
        #alpha = max(min(alpha,0.99), 0.001)
        gamma_w = -(1/dt) * log(alpha)
        sigmaperpsq = (1/n) * (Y22 - (2*alpha*Y12) + (alpha**2 * Y11)-
                    (2*phiperp*(1-alpha)*(Y2 - alpha*Y1)) + (n * phiperp**2 * (1-alpha)**2))
        sigma_w = sqrt(max(1e-9, (sigmaperpsq * 2 * gamma_w) / (1 - (alpha**2))))
        phi_w = phiperp + (sigma_w**2) / (2 * gamma_w)
        rmse_list = []
        for i in range(num_scenarios):
            p = generate_cleaned_XOU_path(gamma_w, sigma_w, phi_w, actual_prices[0], horizon, dt, seed=(seed + i))
            rmse_list.append(np.sqrt(np.mean((p[1:] - actual_prices[1:])**2)))                                                                       
        grid_results.append({'Window':w, 'RMSE': np.mean(rmse_list), 'gamma': gamma_w, 'sigma': sigma_w, 'phi':phi_w})
    gdf = pd.DataFrame(grid_results)
    best_row = gdf.loc[gdf['RMSE'].idxmin()]
    opt_w = int(best_row['Window'])
    b_gamma, b_sigma, b_phi = best_row['gamma'], best_row['sigma'], best_row['phi']
    all_paths = []
    f_res = {'rmse': [], 'mae': [], 'mape': [], 'ks': []}
    actual_log_returns = np.diff(np.log(actual_prices + 1e-9))
    for i in range(num_scenarios):
        path = generate_cleaned_XOU_path(b_gamma, b_sigma, b_phi, actual_prices[0], horizon, dt, seed=(seed + i))
        all_paths.append(path)
        err = path[1:] - actual_prices[1:]
        f_res['rmse'].append(np.sqrt(np.mean(err**2)))
        f_res['mae'].append(np.mean(np.abs(err)))
        f_res['mape'].append(np.mean(np.abs(err / (actual_prices[1:] + 1e-9))) * 100)
        _, p_val = ks_2samp(actual_log_returns, np.diff(np.log(path + 1e-9)))
        f_res['ks'].append(p_val)
    all_paths = np.array(all_paths)
    best_path_vals = all_paths[np.argmin(f_res['rmse'])]
    print(f"OPTIMAL WINDOW FOUND: {opt_w} Days")
    print(f"\n[ACTUAL PRICES (Day 0 to {horizon})]:")
    print(np.round(actual_prices, 2).tolist())
    print(f"\n[OPTIMAL PREDICTED PATH (Day 0 to {horizon})]:")
    print(np.round(best_path_vals, 2).tolist())
    plt.figure(figsize=(12, 6))
    p10,p25,p50,p75,p90 = [np.percentile(all_paths, i, axis=0) for i in [10, 25, 50, 75, 90]]
    plt.fill_between(prediction_dates, p10, p90, color = 'purple', alpha = 0.3, label = '90% CI')
    plt.fill_between(prediction_dates, p25, p75, color = 'purple', alpha = 0.6, label = '50% CI')
    plt.plot(prediction_dates, p50, color='navy', ls='--', label='Median Expectation')
    plt.plot(prediction_dates, actual_prices, color='red', lw=2.5, label='Actual Prices')
    plt.xlabel('Date', fontsize=14)
    ax = plt.gca()
    ax.tick_params(axis='x', labelsize=15)
    plt.ylabel('Closing Prices (Ug Shs)', fontsize=14)
    ax = plt.gca()
    ax.tick_params(axis='y', labelsize=15)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%Y'))
    plt.gcf().autofmt_xdate(); plt.legend(loc='upper left'); plt.grid(True, alpha=0.4); plt.show()
    all_horizon_summaries[horizon] = {
        'Opt_Window': opt_w,
        'RMSE': (np.mean(f_res['rmse']), np.percentile(f_res['rmse'], 2.5), np.percentile(f_res['rmse'], 97.5)),
        'MAE': (np.mean(f_res['mae']), np.percentile(f_res['mae'], 2.5), np.percentile(f_res['mae'], 97.5)),
        'MAPE': (np.mean(f_res['mape']), np.percentile(f_res['mape'], 2.5), np.percentile(f_res['mape'], 97.5)),
        'KS_Pass': np.mean(np.array(f_res['ks']) > 0.05) * 100,
        'gamma': b_gamma, 'sigma': b_sigma, 'phi': b_phi
    }
print("\n" + "="*165)
print(f"{'Horizon':<8}|{'Win':<4}|{'sigma':<8}|{'gamma'}|{'phi'}|{'RMSE [95% CI]':<20}|{'MAE [95% CI]':<20}|{'MAPE [95% CI]':<20}|{'KS Pass'}")
print("\n" + "="*165)
for h in prediction_horizons:
    res = all_horizon_summaries[h]
    r_s = f"{res['RMSE'][0]:.2f}[{res['RMSE'][1]:.1f}-{res['RMSE'][2]:.1f}]"
    m_s = f"{res['MAE'][0]:.2f}[{res['MAE'][1]:.1f}-{res['MAE'][2]:.1f}]"
    p_s = f"{res['MAPE'][0]:.2f}[{res['MAPE'][1]:.1f}-{res['MAPE'][2]:.1f}]"
    print(f"{h:<8}|{res['Opt_Window']:<4}|{res['sigma']:<8.4f}|{res['gamma']:<8.4f}|{res['phi']:<8.4f}|{r_s:<25}|{m_s:<25}|{p_s:25}|{res['KS_Pass']:.1f}%")
print("=" * 165)      
        